### Cointegration Overview
https://www.youtube.com/watch?v=q5wbOSjbVW4

Finds if for two stocks $x_t$ and $y_t$, \
    if $y_t - \beta x_t$ is stationary

Fit linear regression, find estimators $\alpha$ and $\beta$ such that \
    $y_t = \alpha+ \beta x_t + u_t$, where $u_t$ is the residuals \
    then $u_t$ should be stationary

Then try to find $u_t = \delta_0 + \delta_1 u_{t-1} + \delta_2 u_{t-2} + ...$ \
    find if $\delta_1$ does not follow a Dickey-Fuller distribution, \
    if so we can reject the null and say that we have found a stationary pair

But, it is scuffed because this is assuming $\beta$ is correct, when we just estimated it \
    instead, we can use some other more stringent Dickey-Fuller distribution

TLDR: just run the test tbh, it just gives a p-value for pairs trading, may be scuffed

In [27]:
""" 
Imports
"""

import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import coint
from itertools import combinations
from tqdm import tqdm

In [28]:
""" 
Stocks df is already saved to hourlyData.csv now,
    and original Gio data is deleted
    + only tickers ending with USDT saved

Loads data from saved csv
"""

# Old params
start_time = datetime(2024, 6, 1, 0, 0)
end_time = datetime(2024, 8, 25, 0, 0)
test_end_time = datetime(2024, 10, 7, 0, 0)

# New params
start_time = datetime(2024, 8, 1, 0, 0)
end_time = datetime(2024, 10, 7, 0, 0)
test_end_time = datetime(2024, 10, 30, 0, 0)


stocksDf = pd.read_csv('./crypto_data.csv')
stocksDf['time'] = pd.to_datetime(stocksDf['time'])
stocksDf = stocksDf.set_index('time')
stocksDf = stocksDf.loc[start_time:end_time]

stocksDf = stocksDf.dropna(how='all', axis=0)
stocksDf = stocksDf.dropna(how='all', axis=1)
display(stocksDf.head())
display(stocksDf.tail())

closeDf = stocksDf.filter(regex='_close$')
closeDf.columns = closeDf.columns.str.replace('_close$', '', regex=True)
display(closeDf.head())

,ARKUSDT_open,ARKUSDT_high,ARKUSDT_low,ARKUSDT_close,ARKUSDT_volume,ARKUSDT_turnover,AXLUSDT_open,AXLUSDT_high,AXLUSDT_low,AXLUSDT_close,...,ZRXUSDT_low,ZRXUSDT_close,ZRXUSDT_volume,ZRXUSDT_turnover,GRTUSDT_open,GRTUSDT_high,GRTUSDT_low,GRTUSDT_close,GRTUSDT_volume,GRTUSDT_turnover
time,,,,,,,,,,,,,,,,,,,,,
2024-08-01 00:00:00,0.3707,0.3724,0.3598,0.3601,470894.0,171571.5459,0.6015,0.6035,0.5862,0.5924,...,0.3470,0.3485,2261071.0,795634.6694,0.17791,0.17871,0.17469,0.17529,3796879.1,669615.935890
2024-08-01 01:00:00,0.3601,0.3659,0.3569,0.3654,283968.0,102707.8402,0.5924,0.6047,0.5892,0.6047,...,0.3465,0.3565,515127.0,181777.1563,0.17529,0.17774,0.17411,0.17755,2715774.2,477782.508575
2024-08-01 02:00:00,0.3654,0.3654,0.3583,0.3583,198827.0,71892.1171,0.6047,0.6048,0.5960,0.5960,...,0.3496,0.3496,315193.0,111180.0287,0.17755,0.17760,0.17536,0.17536,460018.2,81199.026330
2024-08-01 03:00:00,0.3583,0.3631,0.3582,0.3610,152566.0,55000.6148,0.5960,0.6022,0.5960,0.6004,...,0.3496,0.3533,825604.0,292287.0627,0.17536,0.17695,0.17531,0.17645,625552.3,110324.103009
2024-08-01 04:00:00,0.3610,0.3634,0.3598,0.3633,74562.0,26949.8906,0.6004,0.6045,0.5994,0.6018,...,0.3533,0.3608,428015.0,152983.9286,0.17645,0.17784,0.17624,0.17765,717200.0,127143.566845


,ARKUSDT_open,ARKUSDT_high,ARKUSDT_low,ARKUSDT_close,ARKUSDT_volume,ARKUSDT_turnover,AXLUSDT_open,AXLUSDT_high,AXLUSDT_low,AXLUSDT_close,...,ZRXUSDT_low,ZRXUSDT_close,ZRXUSDT_volume,ZRXUSDT_turnover,GRTUSDT_open,GRTUSDT_high,GRTUSDT_low,GRTUSDT_close,GRTUSDT_volume,GRTUSDT_turnover
time,,,,,,,,,,,,,,,,,,,,,
2024-10-06 20:00:00,0.4878,0.4965,0.4826,0.4848,7952350.0,3.891064e+06,0.6637,0.6906,0.6597,0.6901,...,0.3207,0.3251,295775.0,95585.5566,0.16415,0.16934,0.16305,0.16934,6896535.0,1.143428e+06
2024-10-06 21:00:00,0.4848,0.4934,0.4778,0.4884,4188028.0,2.035001e+06,0.6901,0.6975,0.6808,0.6936,...,0.3246,0.3268,222535.0,72476.7973,0.16934,0.17126,0.16925,0.17099,3504640.8,5.968976e+05
2024-10-06 22:00:00,0.4884,0.4974,0.4877,0.4970,3029974.0,1.494903e+06,0.6936,0.6949,0.6701,0.6706,...,0.3262,0.3264,167542.0,54846.8250,0.17099,0.17187,0.16982,0.17178,4145085.1,7.086339e+05
2024-10-06 23:00:00,0.4970,0.5043,0.4907,0.4947,5712865.0,2.840679e+06,0.6706,0.6718,0.6597,0.6697,...,0.3249,0.3264,146279.0,47695.9096,0.17178,0.17350,0.17077,0.17291,4339655.1,7.460562e+05
2024-10-07 00:00:00,0.4947,0.5067,0.4925,0.5053,4136915.0,2.068239e+06,0.6697,0.6731,0.6654,0.6663,...,0.3252,0.3255,41718.0,13618.7169,0.17291,0.17342,0.17131,0.17143,2824283.8,4.871851e+05


,ARKUSDT,AXLUSDT,IOUSDT,HOOKUSDT,STORJUSDT,ASTRUSDT,SFPUSDT,HNTUSDT,ARUSDT,HOTUSDT,...,ONDOPERP,RAYDIUMUSDT,FUNUSDT,HMSTRUSDT,RENUSDT,PENDLEUSDT,REQUSDT,SAGAUSDT,ZRXUSDT,GRTUSDT
time,,,,,,,,,,,,,,,,,,,,,
2024-08-01 00:00:00,0.3601,0.5924,2.335,0.4973,0.4363,0.068450,0.7572,4.728,28.483,0.001565,...,0.9266,2.1211,0.003383,NaN,0.04189,3.4002,0.09770,1.3350,0.3485,0.17529
2024-08-01 01:00:00,0.3654,0.6047,2.371,0.5042,0.4424,0.069105,0.7640,4.801,28.719,0.001595,...,0.9364,2.1554,0.003432,NaN,0.04292,3.4531,0.09938,1.3698,0.3565,0.17755
2024-08-01 02:00:00,0.3583,0.5960,2.339,0.4965,0.4380,0.068120,0.7614,4.759,28.685,0.001574,...,0.9337,2.1410,0.003387,NaN,0.04222,3.3773,0.09806,1.3294,0.3496,0.17536
2024-08-01 03:00:00,0.3610,0.6004,2.342,0.4967,0.4370,0.068260,0.7627,4.764,28.746,0.001582,...,0.9239,2.1694,0.003386,NaN,0.04238,3.3596,0.09867,1.3290,0.3533,0.17645
2024-08-01 04:00:00,0.3633,0.6018,2.349,0.5027,0.4369,0.068715,0.7663,4.756,29.329,0.001600,...,0.9327,2.1939,0.003396,NaN,0.04292,3.4090,0.09949,1.3622,0.3608,0.17765


In [29]:
""" 
Use only low volume stocks (lowest 10%)
"""

tickers = closeDf.columns
volList = []

for ticker in tickers:
    prices = stocksDf[f'{ticker}_open']
    volumes = stocksDf[f'{ticker}_volume']
    combined = prices*volumes
    mean = combined.mean()
    volList.append((ticker, mean))

volList.sort(key=lambda x:x[1])

percentUsed = 10
lowestVolume = [pair[0] for pair in volList[:len(volList)*percentUsed//100]]
lowestVolume

['SOL-29NOV24',
 'MNTPERP',
 'AEVOPERP',
 'ETHFIPERP',
 'ARBPERP',
 'SHIB1000PERP',
 'POLPERP',
 'STRKPERP',
 'ORDIPERP',
 'DOGEPERP',
 'WLDPERP',
 'NOTPERP',
 'ETH-29NOV24',
 'ONDOPERP',
 'ETCPERP',
 'FDUSDUSDT',
 'FUNUSDT',
 'ETH-26SEP25',
 'BTC-26SEP25',
 'USDEUSDT',
 'LTOUSDT',
 'TIAPERP',
 'FORTHUSDT',
 '1000PEPEPERP',
 'GNOUSDT',
 'OPPERP',
 'OSMOUSDT',
 'VTHOUSDT',
 'TONPERP',
 'XRDUSDT',
 'BTC-29NOV24',
 'DEXEUSDT',
 'WIFPERP',
 'SCRTUSDT',
 'BNBPERP',
 'SUIPERP',
 'POPCATPERP',
 'COMBOUSDT',
 'ETH-28MAR25',
 'XNOUSDT',
 'XCNUSDT',
 'ETH-27DEC24',
 'OXTUSDT',
 'DGBUSDT']

In [30]:
# Drop stocks w/o full data
subsetDf = closeDf.loc[:,lowestVolume]
subsetDf = subsetDf.dropna(how='any', axis=1)
subsetDf.head()
len(subsetDf.columns)

34

In [31]:
# Run cointegration
results = []

for col1, col2 in tqdm(list(combinations(subsetDf.columns, 2))):
    coin_result = coint(closeDf[col1], closeDf[col2], trend='c') #coin_result is tuple of statistics
    results.append([coin_result, col1, col2])

    p_value = coin_result[1]

100%|██████████| 561/561 [00:48<00:00, 11.45it/s]


In [32]:
results.sort(key=lambda x:x[0][1]) # sort by increasing p-value

for i in range(20):
    result = results[i]
    p_value = result[0][1]
    stock1, stock2 = result[1], result[2]
    print(stock1, stock2, p_value)

MNTPERP TIAPERP 0.00010527102062586663
ONDOPERP TIAPERP 0.00036103124375812896
ETH-28MAR25 DGBUSDT 0.0007820728609998495
ETHFIPERP TIAPERP 0.0008476137220316887
ETCPERP DGBUSDT 0.001194180174790143
ETH-27DEC24 DGBUSDT 0.0013153321436489129
FUNUSDT FORTHUSDT 0.0013219530889344207
ETHFIPERP ARBPERP 0.0014448000867373688
GNOUSDT DGBUSDT 0.0015176227767746333
ETCPERP TIAPERP 0.001706677532563062
SCRTUSDT OXTUSDT 0.0019229909078925746
ETHFIPERP VTHOUSDT 0.0021902672533278015
COMBOUSDT DGBUSDT 0.002879613972930064
ETHFIPERP SUIPERP 0.003643858512282668
XCNUSDT OXTUSDT 0.0040087573604753336
ETHFIPERP OSMOUSDT 0.004235006497565291
MNTPERP AEVOPERP 0.004390431242956273
MNTPERP OSMOUSDT 0.0056477680105172755
ETHFIPERP DGBUSDT 0.0061512014304157285
ARBPERP TIAPERP 0.0063116404567945365


In [33]:
import statsmodels.api as sm

# See if the pairs seem like they are actually mean reverting
def checkPair(stock1, stock2, graph=True):
    x = closeDf[stock1]
    x_const = sm.add_constant(x)
    y = closeDf[stock2]

    model = sm.OLS(y, x_const)
    r = model.fit()
    const, slope = r.params

    signal = y - (slope*x + const)
    std = np.std(signal)
    print(f"test_statarb('{stock2}', '{stock1}', {slope:.5g}, {const:.5g}, {std:.5g})") # print commands

    if not graph: return
    
    combined = y - (slope*x + const)

    fig, axs = plt.subplots(1, 3)
    fig.set_figwidth(20)
    fig.set_figheight(5)
    fig.tight_layout()

    tick_locs = combined.index[::len(combined)//4]

    combinedPlot = axs[0]
    combinedPlot.plot(combined)
    combinedPlot.set_title(f'{stock2} Close - ({slope:.4g} * {stock1} Close + {const:.4g})')
    combinedPlot.set_xticks(tick_locs, tick_locs, rotation=45)

    stock1Plot = axs[1]
    stock1Plot.plot(x)
    stock1Plot.set_title(f'{stock1} Close')
    stock1Plot.set_xticks(tick_locs, tick_locs, rotation=45)

    stock2Plot = axs[2]
    stock2Plot.plot(y)
    stock2Plot.set_title(f'{stock2} Close')
    stock2Plot.set_xticks(tick_locs, tick_locs, rotation=45)

In [34]:
for i in range(20):
    stock1, stock2 = results[i][1], results[i][2]
    checkPair(stock1, stock2, graph=False)

test_statarb('TIAPERP', 'MNTPERP', 14.811, -3.7559, 0.46739)
test_statarb('TIAPERP', 'ONDOPERP', 7.0865, 0.19759, 0.43472)
test_statarb('DGBUSDT', 'ETH-28MAR25', 1.3735e-06, 0.0030763, 0.00035212)
test_statarb('TIAPERP', 'ETHFIPERP', 3.2837, 0.36726, 0.46345)
test_statarb('DGBUSDT', 'ETCPERP', 0.00036568, -0.00015874, 0.00030344)
test_statarb('DGBUSDT', 'ETH-27DEC24', 1.4405e-06, 0.0029837, 0.00035144)
test_statarb('FORTHUSDT', 'FUNUSDT', 1014.2, -0.43762, 0.060189)
test_statarb('ARBPERP', 'ETHFIPERP', 0.27085, 0.1614, 0.016934)
test_statarb('DGBUSDT', 'GNOUSDT', 2.8131e-05, 0.0022015, 0.00029619)
test_statarb('TIAPERP', 'ETCPERP', 0.53336, -4.9412, 0.4869)
test_statarb('OXTUSDT', 'SCRTUSDT', 0.24485, 0.02125, 0.0033312)
test_statarb('VTHOUSDT', 'ETHFIPERP', 0.00070545, 0.00096964, 6.5433e-05)
test_statarb('DGBUSDT', 'COMBOUSDT', 0.010374, 0.0024461, 0.0002534)
test_statarb('SUIPERP', 'ETHFIPERP', 0.9008, -0.22335, 0.35693)
test_statarb('OXTUSDT', 'XCNUSDT', 48.514, 0.0074047, 0.002562